# Notebook E-tivity 3 CE4021 Task 2

**Student name:** Jason Coleman

**Student ID:** 9539719

<hr style=\"border:2px solid gray\"> </hr>

## Imports

In [54]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style=\"border:2px solid gray\"> </hr>

## Task 2

Use the below information to create a Naive Bayes SPAM filter. Test your filter using the messages in new_emails. You may add as many cells as you require to complete the task.

In [55]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

<hr style=\"border:2px solid gray\"> </hr>

## Introduction

This will be a very naive Naive Bayes classifier. It will follow a so-called `Bag-of-words` approach where I will classify a message as `SPAM` or `HAM` based on the presence of the words in the string. Notably, the bag of words does not consider the context in which the work is used (i.e. you can reorder the words into complete nonsense and get the same result). 

## Background

### Bayes Theorem
Let us start with the formula for Bayes Theorem. 

$$
P(A \mid B) = \frac{P(B \mid A) \times P(A)}{P(B)}
$$

* where $P(A \mid B)$ is the probability of event A happening given that event B has happened.
* and $P(B \mid A)$ is the probability of event B given A.
* and $P(A)$ and $P(B)$ are the probabilities of events A and B, respectively.

In the context of our spam filter:


*  where $A$ could be the event "The email is spam".
*  and $B$ could be "The email contains the word 'pAssword'".

### Naive Bayes Classifier (NB)

NB is a classification technique based on Bayes' theorem; with the "naive" assumption that every pair of features is independent of each other. Even though it is caled naive, it can perform surpisingly well. I worked as an analysts in the AV industry a few years ago and, in the early days, when we had less data (and features) NB worked really well. 

Reframing this a little, in the context of SPAM/HAM.

* Where $S$ represents the event that an email is spam.
* and $W$ represents the event that a specific word appears in the email.

We can write:

$$
P(S \mid W) = \frac{P(W \mid S) \times P(S)}{P(W)}
$$


*  Where $P(S \mid W)$ is the probability that an email is spam given that it contains the word $W$.
*  and $P(W \mid S)$ is the probability that the word $W$ appears in a spam email.
*  and $P(S)$ is the overall probability (or prior) that any email is spam. This is a `prior`. 
*  and $P(W)$ is the probability that the word $W$ appears in any email, regardless of it being spam or not. This is a `prior`. 

#### Working on messages with more than one word

Emails are rarely composed of just one word so our implemnentation will need to account for all of the words in the message. So, for a given email message, $m$, consisting of words $w_1$, $w_2$, $\ldots$, $w_n$, we need to modify the method to account for multiple words. 

0. We will need to calculate the likelihoods for words being both SPAM or HAM. That is: $P(w_1 | S)$, $P(w_1 | H)$ for each word int he training data. 

Then, for the training messages:

1. Calculate the likelihood of the email being Spam, based on word frequncy:

$$
P(m | S) = P(w_1 | S) \times P(w_2 | S) \times \ldots \times P(w_n | S)
$$

2. Calculate the likelihood of the email being Ham, based on word frequncy:

$$
P(m | H) = P(w_1 | H) \times P(w_2 | H) \times \ldots \times P(w_n | H)
$$

Then, finally, calculate the posterior probabilitites for being Ham and Spam.

$$
P(S | m) = \frac{P(m | S) \times P(S)}{P(m)}
$$

and

$$
P(H | m) = \frac{P(m | H) \times P(H)}{P(m)}
$$

* Where $P(S)$ and $P(H)$ are the prior probabilities of a message being spam and ham, respectively (we know this from the initial training set).
* and $P(m)$ is the total probability of observing message $m$. 

Note, I don't need to calculate $P(m)$ if I am comparing the relative probability of $P(S | m)$ and $P(H | m)$ so the method I will implement will actually be the following:

$$
P(S | m) = P(w_1 | S) \times P(w_2 | S) \times \ldots \times P(w_n | S) \times P(S)
$$

and

$$
P(H | m) = P(w_1 | H) \times P(w_2 | H) \times \ldots \times P(w_n | H)\times P(H)
$$

#### Classification
Then, given a new message, if $P(S | m) > P(H | m)$, I will classify the message as SPAM, else I will classify the message as Ham. 

By doing this, I am side-stepping using a threshold and just comparing the relative posterior probabilitites for the message being HAM and SPAM.

## Python Implementation
We will need to preprocess the data a little; to get it into the right shape. We will start by getting the components of the Bayes formula: the `priors` and the `likelihoods` for the words within the messages. 

**Tokenise:** break each message into words. Each word is called a `token`. It's the fundemental unit of classification in a Bag of Words classifier.

We will need to calculate Priors and likelihoods. To do this we first need to construct a vocabulary. 

**Notes**:

* I will reference the steps in the reference implementation
* My implementation will use list and dictionary comprehensions to make the code more concise.
* I will print intermediate output to aid understanding and debugging.

### Build a vocabulary
The prior probability of an email being spam or ham is calculated based on the proportion of spam or ham emails in the training data.

1. Create a set called *vocabulary*.
2. For each email in the SPAM and HAM training messages, split the words
3. Add all words to the set.

We now have a vocabulary.

In [56]:
vocabulary = set()

for email in previous_spam + previous_ham:
    words = email.split()
    for word in words:
        vocabulary.add(word)

## Calculate the Priors

The prior probability of an email being spam or ham is calculated based on the proportion of spam or ham emails that exists in the training data.

$$
\texttt{prior\_spam} = \frac{\texttt{num\_spam}}{\texttt{total}}
$$

and

$$
\texttt{prior\_ham} = \frac{\texttt{num\_ham}}{\texttt{total}}
$$

In [57]:
num_spam = len(previous_spam)
num_ham = len(previous_ham)

total = num_spam + num_ham

prior_spam = num_spam / total
prior_ham = num_ham / total

print(f"Prior Spam: {prior_spam}")
print(f"Prior Ham: {prior_ham}")

Prior Spam: 0.5714285714285714
Prior Ham: 0.42857142857142855


## Calculate the Conditional Probabilitites (aka Likelihoods)
Here, for each word in the vocabulary, the probability of the word given that an email is spam or ham is calculated. We do this by:

1. Counting the occurrences of each word in both the spam and ham emails.
2. Apply Laplace Smoothing to avoid zero probabilities. If we don't add this, we will eventually end up multiplying by zero (as the word has never been seen before, the conditional probability willk be zero).
3. Calculate the conditional probabilities (likelihoods) for each word being both SAPM and HAM.

In [58]:
# Create dictionaries from the vocabulary to store word counts, initialise the count to zero
spam_word_counts = {word: 0 for word in vocabulary}
ham_word_counts = {word: 0 for word in vocabulary}
print(f"Vocabulary: \n{vocabulary}\n")

# Determine frequencies of words in spam and ham emails (I cosider repeatred words here)
for email in previous_spam:
    words = email.split()
    for word in words:
        spam_word_counts[word] += 1

for email in previous_ham:
    words = email.split()
    for word in words:
        ham_word_counts[word] += 1

print("Spam/HAM dictionaries are as follows:\n")

# print the dictionaries for spam ham in one table
print('\tWord\t\tSpam Count \tHam Count')
print('\t'+'-' * 40)
for word in vocabulary:
    print(f'\t{word:10s}\t{spam_word_counts[word]:2d}\t\t{ham_word_counts[word]:2d}')

print()

#  Preparing the (smoothed) likelihoods
alpha = 1 # Laplace smoothing (in this case 'add-one' smoothing) parameter

spam_total_words = (sum(spam_word_counts.values()) + alpha) * len(vocabulary)
ham_total_words = (sum(ham_word_counts.values()) + alpha) * len(vocabulary)

# Compute the likeloods for SPAM/HAM.
spam_likelihoods = {word: (count + alpha) / spam_total_words for word, count in spam_word_counts.items()}
ham_likelihoods = {word: (count + alpha) / ham_total_words for word, count in ham_word_counts.items()}

print("The condtional probabilities (likelihoods) for spam and ham are as follows:\n")

# print a table of spam_probs and ham_probs
print('\tWord\t\tP(word|spam)\tP(word|ham)')
print('\t'+'-' * 40)
for word in vocabulary:
    print(f'\t{word:10s}\t{spam_likelihoods[word]:.4f}\t\t{ham_likelihoods[word]:.4f}')
    


Vocabulary: 
{'send', 'Your', 'physical', 'report', 'benefits', 'importance', 'us', 'our', 'activity', 'account', 'website', 'the', 'vows', 'password', 'your', 'review'}

Spam/HAM dictionaries are as follows:

	Word		Spam Count 	Ham Count
	----------------------------------------
	send      	 3		 0
	Your      	 0		 1
	physical  	 0		 1
	report    	 0		 1
	benefits  	 0		 1
	importance	 0		 1
	us        	 2		 0
	our       	 1		 0
	activity  	 0		 2
	account   	 1		 0
	website   	 1		 0
	the       	 0		 1
	vows      	 0		 1
	password  	 2		 0
	your      	 3		 0
	review    	 1		 0

The condtional probabilities (likelihoods) for spam and ham are as follows:

	Word		P(word|spam)	P(word|ham)
	----------------------------------------
	send      	0.0167		0.0063
	Your      	0.0042		0.0125
	physical  	0.0042		0.0125
	report    	0.0042		0.0125
	benefits  	0.0042		0.0125
	importance	0.0042		0.0125
	us        	0.0125		0.0063
	our       	0.0083		0.0063
	activity  	0.0042		0.0187
	account   	0.0083		

## Test the Classifier
We can now test the classfier on the test set of data. We use the previously calculated priors and likelihood to calculate the posterior probabilities for both SPAM and HAM and this will allow me to classify new emails as spam or ham. The classification is done by comparing the posterior probability for Ham to that of SPAM: the highest one is the output label.

We can assess the accuracy of the classifier against data it has not seen before. We do this with a labelled test set.

In [59]:
num_correct = 0
num_total = 0

# for all labelled messages in the test set, grouped by label
for label, emails in new_emails.items():
    for email in emails:
        words = email.split()

        # initialise with the priors
        spam_posterior_prob = prior_spam
        ham_posterior_prob = prior_ham
        
        # Caculate the posterior probabilities based on the multiplied likelihoods
        for word in words:
            if word in spam_likelihoods:
                spam_posterior_prob *= spam_likelihoods[word]
            else:
                spam_posterior_prob *= alpha / spam_total_words
            if word in ham_likelihoods:
                ham_posterior_prob *= ham_likelihoods[word]
            else:
                ham_posterior_prob *= alpha / ham_total_words
                
        # I think it is better to compare the relative probabilities instead of
        # picking a threshold. You always get into a trade-off between false
        # positives and false negatives.        
        if spam_posterior_prob > ham_posterior_prob:
            predicted_label = 'spam'
        else:
            predicted_label = 'ham'

        print(f'{email:30s} {predicted_label}')

        if predicted_label == label:
            num_correct += 1

        num_total += 1

accuracy = num_correct / num_total
print(f'\nAccuracy: {accuracy:.2f}')

renew your password            spam
renew your vows                ham
benefits of our account        ham
the importance of physical activity ham

Accuracy: 0.75


## How does this work?
To understand how this works, let's take an example message, `renew your password` from the SPAM test set. Let's start with the word `renew`.


$$
P(\text{spam}) = \text{prior\_spam}
$$


$$
P(\text{ham}) = \text{prior\_ham}
$$

And


$$
P(\text{renew} | \text{spam}) = \text{spam\_likelihoods[renew]}
$$


$$
P(\text{renew} \mid \text{ham}) = \text{ham\_likelihoods[renew]}
$$


We do this for every word int he message, multiplying the spam and ham likelihoods for the words int he message. If the word is not present in our training data, we still give it a non-zero probability because of the Laplace smoothing. This ensures our model can handle unseen words (if we dont do this, we will end up multiplying a zero into the spam_posterior_prob and/or ham_posterior_prob term).

Finally, after computing the probabilities for each word, if the summed probability of the email being spam is higher than that of it being ham, the email is classified as spam, and vice versa.

Let's package all of the previous code into classes, so it's more modular.

In [60]:
class NaiveBayesClassifier:
    """
    Create a NB classifier for 2 classes: spam and ham.

    Note that I have tried to break the methods down to reduce what
    my linter calls Conginitive Complexity. This makes it easier to
    understand and test the code.

    I have two prediction methods:
    - predict: returns the label of the email based on relative probabilities
    - predict_t: returns the label of the email based on a threshold
    """

    def __init__(self) -> None:
        self.__vocabulary = set()
        self.__spam_word_counts = {}
        self.__ham_word_counts = {}
        self.__spam_likelihoods = {}
        self.__ham_likelihoods = {}
        self.__spam_prior = 0
        self.__ham_prior = 0
        self.__alpha = 1


    def __repr__(self) -> str:
        """
        This just helps me pretty print the class 
        to help understand the internals a little better
        
        :return: a string representation of the classifier
        """
        return (f'NB Classifier with {len(self.__vocabulary)} words.\n'
                f'\tA priori probabilities: spam={self.__spam_prior}, ham={self.__ham_prior}.\n'
                f'\tAlpha parameter: {self.__alpha}')
    

    def fit(self, spam_emails: list, ham_emails: list) -> None:
        """
        Train the classifier with spam and ham emails. 

        Throw an exception if there are no words in the vocabulary

        :param spam_emails: a list of spam emails
        :param ham_emails: a list of ham emails
        :param remove_stop_words: remove stopwords from the emails
        :return: None
        """

        # Step 1
        self.__build_priors(spam_emails, ham_emails)
        # Step 2
        self.__build_laplacian_likelihoods(spam_emails, ham_emails)
        
    
    def __build_priors(self, spam_emails: list, ham_emails: list) -> None:
        """
        Compute the prior probabilities for spam and ham emails.

        :param spam_emails: a list of spam emails
        :param ham_emails: a list of ham emails
        :return: None
        """
        num_spam = len(spam_emails)
        num_ham = len(ham_emails)

        total = num_spam + num_ham

        self.__spam_prior = num_spam / total
        self.__ham_prior = num_ham / total
        

    def __build_word_frequency(self, emails: list, word_counts: dict) -> None:
        """
        Update word counts for the provided emails.
        Update the vocabulary as well.
        
        :param emails: a list of emails
        :param word_counts: a dictionary to store word counts
        :return: None
        """
        for email in emails:
            words = email.split()
            for word in words:
                self.__vocabulary.add(word)
                if word in word_counts:
                    word_counts[word] += 1
                else:
                    word_counts[word] = 1


    def __compute_smoothed_likelihoods(self, spam_total_words: int, ham_total_words: int) -> None:
        """
        Compute smoothed likelihoods for SPAM and HAM, for each word in the vocabulary.

        :param spam_total_words: total number of words in spam emails
        :param ham_total_words: total number of words in ham emails
        :return: None
        """
        for word in self.__vocabulary:
            self.__spam_likelihoods[word] = (self.__spam_word_counts.get(word, 0) + self.__alpha) / spam_total_words
            self.__ham_likelihoods[word] = (self.__ham_word_counts.get(word, 0) + self.__alpha) / ham_total_words 


    def __build_laplacian_likelihoods(self, spam_emails: list, ham_emails: list) -> None:
        """
        Calculate the likelihoods for spam and ham emails using Laplacian smoothing.

        :param spam_emails: a list of spam emails
        :param ham_emails: a list of ham emails
        :return: None 
        """
        self.__build_word_frequency(spam_emails, self.__spam_word_counts)
        self.__build_word_frequency(ham_emails, self.__ham_word_counts)

        self.__alpha = 1
        vocabulary_len = len(self.__vocabulary)

        spam_total_words = sum(self.__spam_word_counts.values()) + self.__alpha * vocabulary_len
        ham_total_words = sum(self.__ham_word_counts.values()) + self.__alpha * vocabulary_len

        if spam_total_words == 0 or ham_total_words == 0:
            raise ValueError('No words in the vocabulary')

        self.__compute_smoothed_likelihoods(spam_total_words, ham_total_words)

    def __compute_probability(self, words: list, likelihoods: dict, prior: float) -> float:
        """
        Compute the probability of a list of words given the likelihoods and prior.
        
        :param words: a list of words
        :param likelihoods: a dictionary of likelihoods
        :param prior: the prior probability
        :return: the probability
        """
        prob = prior
        total_words = sum(likelihoods.values()) + self.__alpha * len(self.__vocabulary)
        for word in words:
            if word in likelihoods:
                prob *= likelihoods[word]
            else:
                prob *= self.__alpha / total_words
        return prob


    def predict(self, email:str) -> str:
        """
        Classify an email as spam or ham

        :param email: the email to classify
        :return: the label of the email
        """

        words = email.split()
        spam_prob = self.__compute_probability(words, self.__spam_likelihoods, self.__spam_prior)
        ham_prob = self.__compute_probability(words, self.__ham_likelihoods, self.__ham_prior)

        return 'spam' if spam_prob > ham_prob else 'ham'
        
    def predict_t(self, email:str, threshold: float=0.66) -> str:
        """
        Classify an email as spam or ham

        :param email: the email to classify
        :param threshold: the threshold to use to classify the email
        :return: the label of the email
        """
        words = email.split()
        spam_prob = self.__compute_probability(words, self.__spam_likelihoods, self.__spam_prior)
        
        return 'spam' if spam_prob > threshold else 'ham'

In [61]:
class ClassifierTester:
    """ 
    The prupose of this class is to test the classifier. If I was doing 
    this properly, I would use a notion of abstract methods so the
    tester could test the accuracy of any approach that uses the same 
    signatures for fit and predict.
    """
    def __init__(self, classifier):
        self.classifier = classifier

    def test(self, test_data, print_results=True):
        """
        Test the classifier with test data. This just measures
        how many correct predictions the classifier makes (divided
        by the total number of possible predictions).     

        :param test_data: a dictionary of test data, where 
                          the key is the label and the value 
                          is a list of emails
        :param print_results: whether to print the results 
                          (highlight mismatches)
        :return: the accuracy of the classifier
        """
        num_correct = 0
        num_total = 0

        max_len = max([len(email) for emails in test_data.values() for email in emails])

        if print_results:
            print(f'\t{"Message":{max_len}} {"Expected":10} {"Predicted":10}')
            print(('\t'*1 + '-' * max_len) + ' ' + '-' * 10 + ' ' + '-' * 10)

        for label, emails in test_data.items():
            for email in emails:
                predicted_label = self.classifier.predict(email)

                if print_results:
                    print(f'\t{email:{max_len}} {label:10} {predicted_label:10} {"WRONG!" if predicted_label != label else ""}')

                if predicted_label == label:
                    num_correct += 1
                num_total += 1

        accuracy = num_correct / num_total
        
        return accuracy


In [62]:
# Train the classifier
nb_classifier = NaiveBayesClassifier()
nb_classifier.fit(previous_spam, previous_ham)

print(f"{nb_classifier}\n")

# Test the classifier
nb_tester = ClassifierTester(nb_classifier)
print("Testing the classifier...\n")

#How good is the classifier?
accuracy = nb_tester.test(new_emails)
print(f'\nAccuracy: {accuracy:.2f}')

NB Classifier with 16 words.
	A priori probabilities: spam=0.5714285714285714, ham=0.42857142857142855.
	Alpha parameter: 1

Testing the classifier...

	Message                             Expected   Predicted 
	----------------------------------- ---------- ----------
	renew your password                 spam       spam       
	renew your vows                     spam       spam       
	benefits of our account             ham        spam       WRONG!
	the importance of physical activity ham        ham        

Accuracy: 0.75


## Reflection

This toy NB bayes approach demonstrates the use or prior knowledge of SPAM/HAM in the classification of new messages. We would constantly update the likehoods as we get new corrected labelled data.

!!!!!!!!MORE TODO HERE!!!!!!!


### Feedback
TODO

### System Design (Advantages/Disadvantages)
TODO

### Using a Threshold
I have included two ways to predict whether a message is SPAM or HAM:

* Comparingthe relative posterior probabilities (picking the highest as the most likely)
* Using a threshold

I prefer the former asthe latter approach raises the question of what threshold should I set; i.e. how good is good enough.

### Stopwords
Removing stop words actually drops the accuracy down to 50/50. 

### False Positives
A few years ago, I worked in the AV industry as a malware analyst and we always had to deal with notions of false positive results (TP, TN, FP, FN); think how many times your AV thought a valid, reputable application was malware. Misclassification is a fact of life with these methods.

### Bags of Words and Context
Note that this approach is naive in many ways. It assumes the features are independance. Consider what happens if we jumble up the words in our training and test data.

In [63]:
previous_spam = ['us send password your', 'website our review', 'your password send', 'send us your account']
previous_ham = ['activity report Your', 'benefits activity physical', 'the vows importance']
new_emails = {'spam': ['your renew  password', 'renew vows your', 'ur enhance performance'], 'ham': ['our benefits of account', 'You need to will harder work', 'the importance of physical activity']}

# Train the classifier
nb_classifier = NaiveBayesClassifier()
nb_classifier.fit(previous_spam, previous_ham)

# Test the classifier
nb_tester = ClassifierTester(nb_classifier)
print("Testing the classifier...\n")

accuracy = nb_tester.test(new_emails)
print(f'\nAccuracy: {accuracy:.2f}')


Testing the classifier...

	Message                             Expected   Predicted 
	----------------------------------- ---------- ----------
	your renew  password                spam       spam       
	renew vows your                     spam       spam       
	ur enhance performance              spam       spam       
	our benefits of account             ham        spam       WRONG!
	You need to will harder work        ham        spam       WRONG!
	the importance of physical activity ham        ham        

Accuracy: 0.67


Same result. This method ignores word order.

## References
* [Naive Bayes, Clearly Explain!!!](https://www.youtube.com/watch?v=O2L2Uv9pdDA&ab_channel=StatQuestwithJoshStarmer)
* [Bayes theorem, the geometry of changing beliefs](https://www.youtube.com/watch?v=HZGCoVF3YvM&ab_channel=3Blue1Brown)
* [Notes on Naive Bayes Classifiers for Spam Filtering](https://courses.cs.washington.edu/courses/cse312/18sp/lectures/naive-bayes/naivebayesnotes.pdf)

## Appendix
